# part5 - 데이터 사전 처리

- 데이터 분석의 정확도는 분석 데이터의 품질에 의해 좌우된다.
- 데이터 품질을 높이기 위해서는 누락데이터, 중복데이터 등 오류를 수정하고 분석 목적에 맞는 변형을 하는 과정이 중요하다.
- 머신러닝 분석 모형에 데이터를 입력하기 전에 반드시 누락데이터를 제거하거나 다른 적절한 값으로 대체를 해줘야한다. 이는 앞서 데이터의 품질과 관련이 있으며, 알고리즘을 왜곡하는 현상이 발생하기 때문이다.

## 누락데이터 처리

In [2]:
import seaborn as sns

In [3]:
df = sns.load_dataset('titanic')

In [4]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


여기서 타이타닉 데이터셋에서 deck 열에 NaN(Not a Number) 값이 있다. 더 자세히 info() 와 value_counts() 함수를 이용해서 알아보자.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [6]:
df['deck'].value_counts(dropna=False)

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64

누락데이터를 찾는 직접적인 방법으로 isnull(), notnull() 메소드가 있다.
- isnull(): 누락데이터면 True를 반환하고, 유효한 데이터가 존재하면 False를 반환한다.
- notnull(): 유효한 데이터가 존재하면 True를 반환하고, 누락데이터라면 False를 반환한다.

In [8]:
df.head().isnull()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [9]:
df.head().notnull()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True


누락데이터의 개수를 구할 때 isnull() 메소드와 notnull() 메소드를 활용할 수 있다.   
isnull() 메소드의 경우 반환되는 값이 참이면 1, 거짓이면 0으로 판별된다. 따라서 isnull() 메소드를 실행하고, sum(axis=0)메소드를 적용하면 참(1)의 합을 구한다.

In [11]:
df.head().isnull().sum(axis=0)

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           3
embark_town    0
alive          0
alone          0
dtype: int64

### 누락데이터 제거

- 누락 데이터가 들어 잇는 열또는 행을 삭제하는 방법이다.
- 열을 삭제하면 분석대상이 갖는 특성(변수)을 제거하고, 행을 삭제하면 분석 대상의 관측값을 제거하게 된다.

In [15]:
# 반복문으로 각 열의 NaN 개수 계산하기
missing_df = df.isnull()
for col in missing_df.columns:
    missing_count = missing_df[col].value_counts()
    try:
        print(col,':',missing_count[1]) #isnull() True, 1값만 해당하는 것을 갯수를 표시
    except:
        print(col,':',0)

survived : 0
pclass : 0
sex : 0
age : 177
sibsp : 0
parch : 0
fare : 0
embarked : 2
class : 0
who : 0
adult_male : 0
deck : 688
embark_town : 2
alive : 0
alone : 0


여기서는 전체 891명 중에서 688명의 데크 데이터가 누락이 되어있다. 누락데이터가 차지하는 비율이 매우 높기 때문에 데크열의 누락데이터를 삭제하여 분석에 제외하는 것이 의미가 있다.

dropna() 메소드를 사용하여 누락값을 제거를 할 것이고, thread=500 옵션을 적용하여 NaN값을 500개 이상 갖는 모든 열을 삭제한다는 뜻이다. 결국 'deck'열만 삭제가 되는 것이다.

In [17]:
df_dropcol = df.dropna(axis=1, thresh=500)
print(df.columns)
print(df_dropcol.columns)

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')
Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')


그전과 이후를 보았을 때 해당하는 조건에 열만 제거가 된것을 볼 수 있다.

또한 891명 승객 중에서 177명은 나이에 대한 데이터가 없다. 승객의 나이가 데이터 분석의 중요한 변수라면, 나이 데이터가 없는 승객의 레코드(행)를 제거하는 것이 좋다.   
dropna() 메소드에 subset을 'age'열로 한정하면 'age'열의 행 중에서 NaN값이 있는 모든 행(axis=0)을 삭제한다. 기본값으로 how='any'옵션이 적용이 되는데, 이는 NaN값이 하나라도 존재하면 삭제한다는 뜻이다.'all'옵션은 행에 모든데이터가 NaN값일 경우에만 삭제를 한다는 것이다.

In [20]:
df_age = df.dropna(subset=['age'], how='any',axis=0)
print(len(df_age))

714
